## Önce Routing Ağı'nı inceleyelim:

* Bu modül x (B,C,H,W) alır, a (B,K) döndürür. a her örnek için “K kernel karışım oranı”dır.

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RoutingMLP(nn.Module):
    def __init__(self, cin: int, K: int = 4, reduction: int = 4, temperature: float = 1.0):
        super().__init__()
        hidden = max(1, cin // reduction)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(cin, hidden)
        self.fc2 = nn.Linear(hidden, K)
        self.temperature = temperature

    def forward(self, x):
        v = self.gap(x).flatten(1)                # (B,C)
        h = F.relu(self.fc1(v))                   # (B,hidden)
        logits = self.fc2(h)                      # (B,K)
        a = F.softmax(logits / self.temperature, dim=1)
        return a


In [26]:
x = torch.randn(2, 16, 32, 32)
router = RoutingMLP(cin=16, K=4, reduction=4, temperature=1.0)
0
a = router(x)
print("a shape:", a.shape)        # (2,4)
print("row sums:", a.sum(dim=1))  # ~tensor([1., 1.])


a shape: torch.Size([2, 4])
row sums: tensor([1.0000, 1.0000], grad_fn=<SumBackward1>)


## 2) Routing’i Dynamic Conv’a bağlama (okunur “eğitim” versiyonu)

Burada K adet kernel var, routing ağı a üretir, sonra:

>**W(x)=k∑​ak​(x)Wk​**

In [27]:
class DynamicConv2d(nn.Module):
    """
    Okunur DynamicConv2d:
      - K uzman kernel saklar
      - Routing ağı ile a (B,K) üretir
      - Wdyn (B, cout, cin, k, k) üretir
      - Her sample için conv uygular (anlamak için en net yol)
    """
    def __init__(self, cin, cout, k=3, stride=1, padding=1, K=4, reduction=4, temperature=1.0):
        super().__init__()
        self.cin, self.cout = cin, cout
        self.k = k
        self.stride = stride
        self.padding = padding
        self.K = K

        # Kernel bankası: (K, cout, cin, k, k)
        self.weight = nn.Parameter(torch.randn(K, cout, cin, k, k) * 0.02)

        # Routing ağı
        self.router = RoutingMLP(cin=cin, K=K, reduction=reduction, temperature=temperature)

    def forward(self, x, return_routing=False):
        B, C, H, W = x.shape
        a = self.router(x)  # (B,K)

        # Wdyn: (B, cout, cin, k, k)
        Wdyn = torch.einsum("bk,kocij->bocij", a, self.weight)

        # Per-sample conv (net ve anlaşılır)
        outs = []
        for i in range(B):
            yi = F.conv2d(x[i:i+1], Wdyn[i], stride=self.stride, padding=self.padding)
            outs.append(yi)

        y = torch.cat(outs, dim=0)
        return (y, a) if return_routing else y


In [28]:
x = torch.randn(2, 16, 32, 32)
dyn = DynamicConv2d(cin=16, cout=32, k=3, padding=1, K=4)

y, a = dyn(x, return_routing=True)
print("y:", y.shape)     # (2,32,32,32)
print("a:", a.shape)     # (2,4)
print("row sums:", a.sum(dim=1))


y: torch.Size([2, 32, 32, 32])
a: torch.Size([2, 4])
row sums: tensor([1.0000, 1.0000], grad_fn=<SumBackward1>)


## Tam hatasız sürüm 

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ---------------------------
# Routing ağı (gating)
# ---------------------------
class RoutingMLP(nn.Module):
    """
    x (B,C,H,W) -> GAP -> (B,C) -> MLP -> logits (B,K) -> softmax -> a (B,K)
    """
    def __init__(self, cin: int, K: int = 4, reduction: int = 4, temperature: float = 1.0):
        super().__init__()
        hidden = max(1, cin // reduction)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(cin, hidden)
        self.fc2 = nn.Linear(hidden, K)
        self.temperature = temperature

    def forward(self, x):
        v = self.gap(x).flatten(1)  # (B,C)
        h = F.relu(self.fc1(v))     # (B,hidden)
        logits = self.fc2(h)        # (B,K)
        a = F.softmax(logits / self.temperature, dim=1)
        return a


# ---------------------------
# Dynamic Convolution
# ---------------------------
class DynamicConv2d(nn.Module):
    """
    K uzman kernel + routing ağı ile input'a özel kernel üretir.
    Okunurluk için per-sample conv döngüsü kullanır (eğitim/demo için ideal).
    """
    def __init__(self, cin, cout, k=3, stride=1, padding=1,
                 K=4, reduction=4, temperature=1.0, bias=False):
        super().__init__()
        self.cin, self.cout = cin, cout
        self.k = k
        self.stride = stride
        self.padding = padding
        self.K = K

        # Kernel bankası: (K, cout, cin, k, k)
        self.weight = nn.Parameter(torch.randn(K, cout, cin, k, k) * 0.02)

        # (Opsiyonel) bias bankası: (K, cout)
        self.bias_bank = nn.Parameter(torch.zeros(K, cout)) if bias else None

        # Routing
        self.router = RoutingMLP(cin=cin, K=K, reduction=reduction, temperature=temperature)

    def forward(self, x, return_routing=False):
        B, C, H, W = x.shape
        a = self.router(x)  # (B,K)

        # Dinamik kernel: Wdyn (B, cout, cin, k, k)
        Wdyn = torch.einsum("bk,kocij->bocij", a, self.weight)

        # Dinamik bias: bdyn (B, cout)
        bdyn = None
        if self.bias_bank is not None:
            bdyn = torch.einsum("bk,kc->bc", a, self.bias_bank)

        outs = []
        for i in range(B):
            yi = F.conv2d(
                x[i:i+1],
                Wdyn[i],
                bias=None if bdyn is None else bdyn[i],
                stride=self.stride,
                padding=self.padding
            )
            outs.append(yi)

        y = torch.cat(outs, dim=0)
        return (y, a) if return_routing else y


# ---------------------------
# Hızlı test
# ---------------------------
if __name__ == "__main__":
    x = torch.randn(2, 16, 32, 32)
    dyn = DynamicConv2d(cin=16, cout=32, k=3, padding=1, K=4, temperature=1.0)

    y, a = dyn(x, return_routing=True)
    print("y shape:", y.shape)       # (2,32,32,32)
    print("a shape:", a.shape)       # (2,4)
    print("row sums:", a.sum(dim=1)) # ~[1, 1]


y shape: torch.Size([2, 32, 32, 32])
a shape: torch.Size([2, 4])
row sums: tensor([1., 1.], grad_fn=<SumBackward1>)


----
-----

## 1) GAP ne?

* GAP = Global Average Pooling (nn.AdaptiveAvgPool2d((1,1)))

* Input: x shape = (B, C, H, W)

GAP çıkışı: (B, C, 1, 1)
>sonra flatten(1) ile (B, C)

#### Ne yapıyor?
* Her kanal için tüm uzamsal piksellerin ortalamasını alıyor:

#### Neden kullanıyoruz?
* Routing ağı “hangi kernel uygun?” kararını vermek için görüntünün global özetine ihtiyaç duyar.
GAP bunu ucuz ve stabil şekilde sağlar


## 2) hidden ne, neden max kullanılıyor?

Kod:
```python
hidden = max(1, cin // reduction)
```

* hidden = routing MLP’nin ara katman boyutu.

* cin küçükse, cin // reduction 0 çıkabilir (ör: 3 // 4 = 0)

* Linear katmanda 0 boyut olamaz → model patlar.

O yüzden:

* max(1, ...) ile en az 1 garanti ediliyor.

**Amaç:**
* Routing ağı küçük kalsın (hafif) ama geçerli boyutta olsun.

## 3) Temperature (temperature) ne işe yarıyor?

Softmax’a girmeden önce:
```python
a = softmax(logits / temperature)
```


* Temperature, softmax’ın “ne kadar seçici” olacağını ayarlar.

* T ↓ (0.5 gibi) → dağılım keskinleşir
> → model genelde “tek kerneli seçmeye” yaklaşır (one-hot benzeri)

* T ↑ (2.0 gibi) → dağılım yumuşar
>→ kernel’ler daha çok karışır (uniform’a yaklaşır)

Kısaca:

* Küçük T = daha agresif seçim

* Büyük T = daha yumuşak karışım

## 4) Softmax’ın içine koyduğumuz şey ne?

* Softmax’ın içine koyduğun şey: logits.

Kod:
```python
logits = fc2(h)  # (B, K)
a = softmax(logits / T)
```


logits ne?
* Henüz olasılık olmayan “ham skorlar”.

Her örnek için K tane skor:

* logit_1, logit_2, ..., logit_K

>Softmax bunları pozitif ve toplamı 1 olan ağırlıklara çevirir:
Bu yüzden a:

* her elemanı 0-1 arası

* satır toplamı = 1

* “kernel karışım oranı” gibi davranır

## 5) Kernel bankası ve bias bankası ne?
Kernel bankası

Kod:
```python
self.weight: (K, cout, cin, k, k)
```


Bu şu demek:

* K tane ayrı ayrı uzman conv kernel var.

Her biri normal Conv2d ağırlığı gibi düşün:

* W1, W2, ..., WK

* Routing ağı a üretince bu kernel’leri karıştırıp tek efektif kernel yapıyoruz.

### Bias bankası

Kod (opsiyonel):
```python
self.bias_bank: (K, cout)
```


Eğer bias kullanırsan:

* Her kernel için ayrı bias vektörü var: b1, b2, ..., bK

* Routing ağıyla bunlar da karıştırılıp efektif bias oluşturulur.

**Not: CNN’lerde BN kullandığın için çoğu zaman bias zaten gereksiz → o yüzden default bias=False mantıklı.**

## 6) torch.einsum burada ne işe yarıyor?

İki tane kritik einsum var:

### (A) Dinamik kernel üretimi
```python
Wdyn = einsum("bk,kocij->bocij", a, weight)
```


* a: (B, K) → her örnek için kernel ağırlıkları

* weight: (K, cout, cin, k, k) → kernel bankası


### (B) Dinamik bias üretimi
```python
bdyn = einsum("bk,kc->bc", a, bias_bank)
```

* einsum burada “ağırlıklı toplama / karışım” yapıyor.
* Yani routing çıktısını kernel bankasına uyguluyor.

## 7) outs ne? For içinde ne yapılıyor?
Neden for var?

PyTorch’un F.conv2d fonksiyonu:

* batch içindeki her örneğe aynı kernel uygular.

Ama Dynamic Conv’da:

* her örnek için kernel farklı (Wdyn[i])

* PyTorch tek çağrıda “batch’te herkes farklı kernel kullansın” diyemiyor (standart conv).

Bu yüzden eğitim/demo amaçlı:

* örnek örnek conv yapıyoruz.

Kod:
```python
outs = []
for i in range(B):
    yi = F.conv2d(x[i:i+1], Wdyn[i], ...)
    outs.append(yi)
y = torch.cat(outs, dim=0)
```


* outs: her örneğin çıktısını tutan liste

döngü:

* i’inci örnek: x[i:i+1] (shape: 1,C,H,W)

* i’inci kernel: Wdyn[i] (cout,cin,k,k)

* conv sonucu: yi (1,cout,H’,W’)

* sonra hepsini batch olarak birleştiriyoruz: cat → (B, cout, H’, W’)

### Özet:

* outs = “her sample’ın conv çıktısı”
* for döngüsü = “her sample’a kendi kerneliyle conv uygulama”